In [40]:
import pandas as pd
import numpy as np

In [41]:
train_set = pd.read_csv("train_ver2.csv")
test_set = train_set.loc[train_set['fecha_dato']=='2016-04-28', :"segmento"]
#test_set = pd.read_csv("test_ver2.csv")

In [42]:
len(test_set)

928274

In [43]:
drop_list = ['fecha_alta', 'ult_fec_cli_1t', 'canal_entrada', 'cod_prov', 'indrel_1mes', 'conyuemp']
train_set.drop(drop_list, axis=1, inplace=True)
test_set.drop(drop_list, axis=1, inplace=True)

#train_set.loc[:, 'conyuemp'] = train_set['conyuemp'].fillna(0)
#test_set.loc[:, 'conyuemp'] = test_set['conyuemp'].fillna(0)

In [44]:
train_set_505 = train_set.loc[(train_set['fecha_dato']=='2015-03-28')]
train_set_506 = train_set.loc[(train_set['fecha_dato']=='2015-04-28')]
train_set_505.set_index('ncodpers', inplace=True)
train_set_506.set_index('ncodpers', inplace=True)
train_set_june = train_set_505.copy()
train_set_june.loc[:, "ind_ahor_fin_ult1":"ind_recibo_ult1"] = train_set_506.loc[:, "ind_ahor_fin_ult1":"ind_recibo_ult1"] - train_set_505.loc[:, "ind_ahor_fin_ult1":"ind_recibo_ult1"]
train_set_june.dropna(inplace=True)
train_set_june['filter'] = train_set_june.loc[:, "ind_ahor_fin_ult1":"ind_recibo_ult1"].abs().sum(1)
train_set_june = train_set_june[train_set_june['filter']>0]
train_set_june_y = train_set_june.loc[:, "ind_ahor_fin_ult1":"ind_recibo_ult1"].copy().reset_index()
train_set_june_y.loc[:, "ind_ahor_fin_ult1":"ind_recibo_ult1"] = (train_set_june_y.loc[:, "ind_ahor_fin_ult1":"ind_recibo_ult1"]>0).astype(float)
train_set_june = train_set_june.loc[:, :"segmento"]
train_set_june.reset_index(inplace=True)

train_set_june.drop(['fecha_dato'], axis=1, inplace=True)
test_set.drop(['fecha_dato'], axis=1, inplace=True)

In [45]:
product_list = list(train_set.columns)[list(train_set.columns).index("ind_ahor_fin_ult1"):list(train_set.columns).index("ind_recibo_ult1")+1]
def append_last_month(data_set, last_month): 
    return pd.merge(data_set, last_month.loc[:, ['ncodpers']+product_list].drop_duplicates(), how="inner", on='ncodpers')
    
train_set_june = append_last_month(train_set_june, train_set.loc[(train_set['fecha_dato']=='2015-03-28')])
test_set = append_last_month(test_set, train_set.loc[(train_set['fecha_dato']=='2016-03-28')])

In [46]:
province_code = train_set.groupby("nomprov").agg({"renta":np.mean}).sort_values(by='renta', ascending=False).reset_index().reset_index().drop(["renta"], axis=1)
province_code.columns = ['f_nomprov', "nomprov"]

country_code = train_set.groupby('pais_residencia').agg({'renta':np.mean}).sort_values(by='renta', ascending=False).reset_index().reset_index().drop(["renta"], axis=1)
country_code.columns = ['f_pais_residencia', "pais_residencia"]

renta_missing = train_set_june['renta'].mean()

In [47]:
def one_hot_encoding(df, column_name, value_set):
    df1 = df.copy()
    assert len(value_set)>1, "len of value set should >1"
    if len(value_set)==2:
        df1["f_{0}".format(column_name)] = df1[column_name].apply(lambda x: 1 if x==value_set[0] else 0)
    else:
        for v in value_set:
            df1["f_{0}_{1}".format(column_name, v)] = df1[column_name].apply(lambda x: 1 if x==v else 0)
    return df1

In [48]:
import gc

def encode_feature(X):
    X1 = X.copy()
    get_value_set = lambda x: list(train_set[x].drop_duplicates().dropna())
    one_hot_encoding_list = ["ind_empleado", "sexo",  "tiprel_1mes", "indresi", "indext", "indfall", "ind_actividad_cliente", "segmento"]
    for c in one_hot_encoding_list:
        # print c
        X1 = one_hot_encoding(X1, c, get_value_set(c))
        gc.collect()
    copy_feature_list = ['age', 'antiguedad', 'indrel', 'ind_nuevo', 'tipodom']
    for c in copy_feature_list:
        X1['f_'+c] = X1[c]  
    X1 = pd.merge(X1, province_code, how="left", on=['nomprov'])
    X1 = X1.fillna(province_code['f_nomprov'].mean())
    X1 = pd.merge(X1, country_code, how="left", on=['pais_residencia'])
    X1 = X1.fillna(country_code['f_pais_residencia'].mean())
    X1['f_renta'] = (X1['renta'].apply(lambda x: renta_missing if "NA" in str(x) else x)).astype(float)
    X1.drop(one_hot_encoding_list+copy_feature_list+['nomprov', 'pais_residencia', 'renta'], axis=1, inplace=True)
    return X1

train_feature = encode_feature(train_set_june)
test_feature = encode_feature(test_set)

In [49]:
print len(train_feature)
print len(train_set_june_y)
train_feature = train_feature.sort_values(by='ncodpers')
train_set_june_y = train_set_june_y.sort_values(by='ncodpers')
X_train = np.array(train_feature.iloc[:, 1:].astype(float).values)
y_train = np.array((train_set_june_y.iloc[:, 1:]>0).astype(int).values)
X_test = np.array(test_feature.iloc[:, 1:].astype(float).values)
print len(X_test)

37253
37253
923414


In [50]:
from sklearn.ensemble import RandomForestClassifier

clm = RandomForestClassifier(n_estimators=50, verbose=1, criterion='entropy', class_weight='balanced', max_depth=10)
clm = clm.fit(X_train, y_train)

[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    9.0s finished


In [51]:
predict = clm.predict_proba(X_test)

[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:  2.7min finished


In [52]:
arr = []
for p in predict:
    if p.shape[1]==1:
        arr.append(1-p[:,0])
    else:
        arr.append(p[:,1])
ret = np.array(arr).T

In [53]:
np.array((test_feature.loc[:, product_list]!=0).values).sum(0)

array([    78,     16, 561433,    319,  70141,   7617,   8715, 101377,
        33630,    331,   1076,  33188,  72063,  14893,   4565,   7366,
         1996,  44463,  34329,  21459,   2982,  47030,  50912, 110872])

In [54]:
filter1 = np.array((test_feature.loc[:, product_list]==0).values)
ret = ret*filter1
user_id = list(test_set["ncodpers"])

f = open("test_sub.csv", 'w')
f.write("ncodpers,added_products\n")

print len(ret)

for idx,r in enumerate(ret):
    order = [i[0] for i in sorted(enumerate(r), key=lambda x:x[1], reverse=True)]
    select = order[:min(7,len(order))]
    f.write(str(user_id[idx])+","+(" ".join([product_list[s] for s in select]))+'\n')
f.close()

923414


In [55]:
train_set_june.sum()

ncodpers                                                       25582682812
ind_empleado             NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...
pais_residencia          ESESESESESESESESESESESESESESESESESESESESESESES...
sexo                     HVVVHVHHVVVVVVVVHHVVVVHVVVVHVVVVHVHVVVVHVVVVHV...
age                       37 47 52 38 45 49 32 45 47 49 38 46 42 42 39 ...
ind_nuevo                                                             1106
antiguedad                   118    118    118    118    114    118    ...
indrel                                                               37351
tiprel_1mes              AAAAAAAAIAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAIAAAAA...
indresi                  SSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSS...
indext                   NNNNNNNNNNNSNNNNNNNNNNNNNNNSNNNNNNNNNNNNNNNNNN...
indfall                  NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...
tipodom                                                              37253
nomprov                  